# 🦌 Node Label Placement 🏷️

In [ ]:
import asyncio
import copy
import json

import importnb
import ipywidgets as W
import traitlets as T
from IPython.display import display

import ipyelk
from ipyelk.diagram import layout_options

In [ ]:
with importnb.Notebook():
    from __00_Introduction import a_simple_elk_json_example

In [ ]:
if __name__ == "__main__":
    nlp = layout_options.NodeLabelPlacement()
    display(nlp)

ELKJSON build acceptable value for the label placement

In [ ]:
if __name__ == "__main__":
    display(nlp.value)

### Example connecting NodeLabelPlacement widget value to the label layoutOptions

In [ ]:
async def update_label_layout_options(node, options, sizer, node_options=None):
    """Recurse through the nodes and update layout options"""
    if node_options:
        node.pop("width", None)
        node.pop("height", None)
        node["layoutOptions"] = node_options

    for label in node.get("labels", []):
        if "layoutOptions" in label:
            label["layoutOptions"].update(options)
        else:
            label["layoutOptions"] = options

        # removing rigid x and y placement
        label.pop("x", None)
        label.pop("y", None)

        # updating label size with measured values from the DOM
        size = await sizer.measure(label.get("text"))
        label["height"] = size.height
        label["width"] = size.width

    for child in node.get("children", []):
        await update_label_layout_options(child, options, sizer, node_options)

In [ ]:
def an_elk_example_with_custom_node_label_placement(nlp=None, update_options=None):
    update_options = update_options or update_label_layout_options
    elk = a_simple_elk_json_example()
    original_value = copy.deepcopy(elk.value)
    sizer = ipyelk.diagram.ElkTextSizer()
    nlp = nlp or layout_options.NodeLabelPlacement()

    async def update_json():
        elk_json = copy.deepcopy(original_value)
        elk_json["properties"] = {"algorithm": "layered"}
        layoutOptions = {"nodeLabels.placement": nlp.value}
        await update_options(elk_json, layoutOptions, sizer)
        elk.value = elk_json

    def refresh(change=None):
        """Wrapper function for scheduling the async update_json function"""
        asyncio.create_task(update_json())

    nlp.observe(refresh, "value")
    refresh()
    box = W.HBox([elk, nlp], layout={"height": "400px"})
    return box, refresh, nlp, sizer, elk

In [ ]:
if __name__ == "__main__":
    box, refresh, nlp, sizer, elk = an_elk_example_with_custom_node_label_placement(nlp)
    display(box)

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).